In [1]:
import glob
import pandas as pd
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import RBF, WhiteKernel
import numpy as np
import os.path 
from tabulate import tabulate
import json
import extinction
from astropy.time import Time #to time converting
from astropy import units #time correction
from astropy.coordinates import SkyCoord #time correction
import math
%run /home/joao/Documents/pesquisa/templates/functions.ipynb

In [2]:
r_v = 3.1
func = extinction.Fitzpatrick99(r_v)

In [3]:
file = open("trainingnamespca.dat")
lines = file.readlines()
file.close()

training_set = []

for line in lines:
    if line.startswith('#'): continue
    co=line.rstrip().replace('INDEF','Nan').split()

    training_set.append(co[0])
    
print('total sn number:', len(training_set))

for i in range(0, len(training_set)):
    
    training_set[i] = training_set[i][:-4]

total sn number: 92


In [4]:
save_path = '/home/joao/Documents/pesquisa/PreTrain/data_err/'
wavegrid = np.linspace(3000, 9000, 200 + 1)
kernel = 0.5 * (RBF(length_scale=5, length_scale_bounds=(1, 10)) + WhiteKernel(noise_level=0.1, noise_level_bounds=(1e-5,0.5)))

In [5]:
for j in range(0, len(training_set)):
    
    with open('/home/joao/Documents/pesquisa/templates/'+training_set[j]+'.json', "r") as read_file:
        data = json.load(read_file)
    
    mwebv = float(data[training_set[j]]['ebv'][0]['value'])
    redshift = redshift_value(data[training_set[j]])   
    
    print(training_set[j])
    wave = []
    fluxes = []
    fluxes_err = []
    
    sn_specs = glob.glob("./data/"+training_set[j]+"*"+".dat")
    
    if len(sn_specs) > 0:
    
        list_data = [[[],[]] for y in range(0, 201)]
        times = []

        for k in range(0, len(sn_specs)):

            spectable = pd.read_csv(sn_specs[k], sep='\s{1,}', skiprows=[0], dtype=float, header = None, engine='python')

            file = open(sn_specs[k])
            lines = file.readlines()
            file.close()

            t = float(lines[0][1:]) #time of spec

            times.append(t)

            for i in range(0, len(spectable)):

                if spectable[1][i] != 0:

                    list_data[i][0].append([t])
                    list_data[i][1].append(spectable[1][i])

        times = np.array(times)
        for jj in range(0,len(list_data)):

            X_mod = []

            Y_mean_mod = []

            count = 0

            for jjj in range(0,len(list_data[jj][1])):

                if list_data[jj][1][jjj] > 10**(-64):

                    count += 1        

            if count > 4:                     

                for ii in range(0,len(list_data[jj][1])):

                    if list_data[jj][1][ii] > 10**(-64):

                        Y_mean_mod.append(list_data[jj][1][ii])
                        X_mod.append(list_data[jj][0][ii])

                ff = max(Y_mean_mod)
                #print(ff)
                Y_mean_mod = Y_mean_mod/ff

                gp = GaussianProcessRegressor(kernel=kernel, alpha=0.0).fit(X_mod, Y_mean_mod)
                y_mean, y_cov = gp.predict(times[:, np.newaxis], return_cov=True)        

                wave.append(wavegrid[jj])
                fluxes.append(y_mean*ff)
                fluxes_err.append(np.sqrt(np.diag(y_cov))*ff)

        for ii in range(0, len(fluxes[0])):

            tflux = []
            tfluxerr = []

            for jj in range(0, len(fluxes)):

                tflux.append(fluxes[jj][ii])    
                tfluxerr.append(fluxes_err[jj][ii])

            tflux = extinction.remove(func(np.array(wave)*(1+redshift), mwebv*r_v), np.array(tflux))
            tfluxerr = extinction.remove(func(np.array(wave)*(1+redshift), mwebv*r_v), np.array(tfluxerr))             
            
            
            
            
            
            
            temporaryvec = tfluxerr/tflux
            
            #print("media de erros", np.mean(temporaryvec))
            
            if np.mean(temporaryvec) < 0.6:
                print(np.mean(temporaryvec))

                table = []
                for kk in range(0,len(wave)):
                    table.append((wave[kk],tflux[kk], tfluxerr[kk]))

                name_of_file = training_set[j]+str(ii)

                header = '#' + str(times[ii])

                completeName = os.path.join(save_path, name_of_file+".dat")

                f = open(completeName, 'w')
                f.write(header + "\n")
                f.write(tabulate(table, tablefmt="plain"))
                f.close()

0.01502
SN2001V
0.3503075139424995
0.4623406527813246
0.4083025426882406
0.3291325639936671
0.4119062456396005
0.33101171980633415
0.3463770509028046
0.3692483695357645
0.3719320750119639
0.3223492682769366
0.5313703526148252
0.37913620201328907
0.3474035788916229
0.3206185477485647
0.007132
SN2002fk
0.2792692211999114
0.20468840659898346
0.2019503086124727
0.21524930482939736
0.23944783383165358
0.2069332799791444
0.22912577826456235
0.1994833928488896
0.016725
SN2006ax
0.2806185391547864
0.5976084172879884
0.34409234504909525
0.41556569759922085
0.4096736834366999
0.2886050157624521
0.5655078226932465
0.28004676109571275
0.3491540169404464
0.031021
SN2004as
0.4642946366076477
0.5625971055170012
0.45970035181036034
0.48750882940615786
0.5226103316390753
0.583676707293895
0.5118173092941452
0.4682291358006439
0.00149
SN1994D
0.55018699249915
0.396400235650507
0.23536285126420237
0.2447356456558051
0.45986160237032353
0.23591219186545795
0.25867652282765874
0.4676566011696077
0.23087423

0.24613864718330866
0.131280843672768
0.07570843544638953
0.10298137125442831
0.08117817453227513
0.19000258977892476
0.00626
SN1992A
0.015067397898713919
SN2008bc
0.3379443290706445
0.09258867152752077
0.08445114347603848
0.09652237448664858
0.08444701221156127
0.21993229912454398
0.08445401054743708
0.21808151009659502
0.15812289558188714
0.023131
SN2004at
0.4219064187303541
0.40588559780440364
0.4526002776069447
0.4113177501783988
0.5275876964382221
0.4649269159862118
0.4781923739666079
0.4399845960889658
0.41548748172322914
0.01141
SN1999ee
0.03
LSQ12gdj
0.1499121805661065
0.11513686376044469
0.1000041899584914
0.3389768020095038
0.02099
SN2008Z
0.16520210179874206
0.1487646847373676
0.24897513203839292
0.154734492243589
0.18207311576812135
0.5529924644670938
0.30248526041071633
0.5154652228628813
0.14992451782170663
0.20945591021644033
0.4660058288768924
0.2045002442247715
0.031003287895856158
SN2004ef
0.29009644912044474
0.24601619245526435
0.3160939647498848
0.2396449645345837
0

<ipython-input-5-96543dd8ad83>:93: RuntimeWarning: divide by zero encountered in true_divide
  temporaryvec = tfluxerr/tflux


0.5016012815978762
0.1406980833809061
0.11888077337836521
0.11481463358881039
0.11481463358881039
0.35240591967464724
-3.4136837045951857e+50
0.35240591967464724
0.1324727119756996
0.23048178729061364
0.1406980833809061
-4.963363627236218e+54
0.24160239447885762
0.012526589416171241
SN2008hv
0.01744
SN2000dk
0.18174810758898885
0.16131743471490612
0.16532459150291012
0.15767716150123026
0.299808067232738
0.007722
SN2017fgc
0.0239
SN2007qe
0.18600991978916237
0.108271950357774
0.45456806776775477
0.10327525931518383
0.176778776810455
0.1843927107648721
0.1852081495464457
0.25474567390314407
0.10862928715667426
0.022289
SN2006cp
0.1447733251696783
0.17845072917793106
0.11949877190738797
0.48058463500170523
0.16134421950621275
0.12155894636066292
0.2136043926411567
0.29006934786249117
0.006004
SN2003fa
0.20170919632507084
0.3061410555393683
0.5604610749564409
0.3729466341638881
0.2862745835487912
0.46396056675700104
0.4274603046215729
0.32477173577369217
0.35924104591604084
0.332704098784

<ipython-input-5-96543dd8ad83>:93: RuntimeWarning: divide by zero encountered in true_divide
  temporaryvec = tfluxerr/tflux


0.006381
SN2003du
0.1501371893511082
0.595423144233004
0.3900626903876554
0.2629574060733498
0.14760083397446763
0.2692495152208604
0.2901209195873567
0.2329424933803724
0.48287888680533947
0.31733468777898904
0.16165433508932942
0.32111604156495066
0.02077
SN2007bc
0.003699
SN1998aq
0.4891897338800335
0.29786568989247786
0.34799277501103326
0.14651684108225177
0.5910277438151407
0.2874530575427429
0.15719591579971715
0.14446620485798778
0.1837477831248348
0.5571700629359965
0.5035195926327223
0.16830980463665515
0.1454398856148924
0.20320380460503581
0.14951353848267682
0.015501
SN2001bf
0.05856228200982286
0.09878704876047273
0.06084893306533202
0.3261186916964673
0.445355126450778
0.5071682932174514
0.3484056130428771
0.0533725041092864
0.00897
SN1998dh
0.4536272934373965
0.40889443214879984
0.35139267517123723
0.32516635061253313
0.3264081815609796
0.5111310708402493
0.0109
SN1997bq
0.29634327777524877
0.3062928632573585
0.533866114145075
0.4503320172146378
0.3267165037069776
0.392

In [6]:
data

{'SN1997bq': {'schema': 'https://github.com/astrocatalogs/supernovae/blob/d3ef5fc/SCHEMA.md',
  'name': 'SN1997bq',
  'sources': [{'name': '2016A&A...594A..13P',
    'bibcode': '2016A&A...594A..13P',
    'reference': 'Planck Collaboration et al. (2016)',
    'alias': '1'},
   {'name': '2013AJ....146...86T',
    'bibcode': '2013AJ....146...86T',
    'reference': 'Tully et al. (2013)',
    'alias': '2'},
   {'name': '2012AJ....143..126B',
    'bibcode': '2012AJ....143..126B',
    'reference': 'Blondin et al. (2012)',
    'alias': '3'},
   {'name': '2012A&A...538A.120L',
    'bibcode': '2012A&A...538A.120L',
    'reference': 'Lennarz, Altmann, & Wiebusch (2012)',
    'alias': '4'},
   {'name': '2011ApJ...737..103S',
    'bibcode': '2011ApJ...737..103S',
    'reference': 'Schlafly & Finkbeiner (2011)',
    'alias': '5'},
   {'name': '2010ApJ...716..712A',
    'bibcode': '2010ApJ...716..712A',
    'reference': 'Amanullah et al. (2010)',
    'alias': '6'},
   {'name': '2008yCat....1.2024B',
